In [2]:
import pandas as pd
import numpy as np

# READ THE CSV INTO DATAFRAME

df = pd.read_csv('Syngenta/Syngenta_2017/Experiment_dataset.csv')

# np_ar = np.asarray(df)
# df2 = pd.read_csv('Syngenta/Syngenta_2017/Region_dataset.csv')

In [3]:
print(df['Planting date'][[np.random.randn]])
# print(df.columns)
# print(df.Temperature.describe())
# import random
# for i in range(0, 500):
#     print(random.choice(df['Planting date']))

KeyError: 'None of [[<built-in method randn of mtrand.RandomState object at 0x7f6cc00eb048>]] are in the [index]'

In [ ]:
df.Yield.describe()

In [6]:
# CURRENTLY NECESSARY IF: USING 174 ADDITIONAL VARIETY COLUMNS METHOD

# THIS IS A DIFFERENT APPROACH TO THE ABOVE FOUR CELLS, WHERE WE HAVE 174 ADDITIONAL FEATURE COLUMNS
# EACH WITH A 0 (IF IT IS NOT OF THAT VARIETY) OR A 1 (IF IT IS OF THAT VARIETY)

# print(df)
variety_dummies = pd.get_dummies(df.Variety)
# print(dummies)
df = pd.concat([df, variety_dummies], axis=1)


In [ ]:
# OPTIONAL VARIETY DISTRIBUTION ANALYSIS

print(variety_dummies.sum().describe())
print(np.sort(variety_dummies.sum()))
for idx, cl in enumerate(variety_dummies.sum()):
    print(variety_dummies.columns[idx], cl)

In [ ]:
# GOAL OF THIS MODULE:
# Encode the planting date as a season
# NEW GOAL:
# GET DUMMIES FOR SEASONS

# remove the dates that are "."
df = df[~df['Planting date'].str.match("\.")]
plant_date = df['Planting date'].apply(lambda dt: pd.to_datetime(dt))
plant_months = plant_date.apply(lambda dt: dt.month)
season = plant_date.rename("Season")
season = pd.to_datetime(season)
season = season.apply(lambda dt: (dt.month%12 + 3)//3)
# df['Plant date'] = pd.to_datetime(df['Plant date'])
df = pd.concat([df, season], axis=1)

# plant_date = pd.to_datetime(df['Planting date'], infer_datetime_format=True)
# df = df['Planting date'].apply(lambda dt: (dt.month%12 + 3)//3)
# pd.get_dummies(df['Planting date'])


In [ ]:
# ADD MONTH OF MAY AND JUNE ONE HOT ENCODING INTO THE DATAFRAME
pd.get_dummies(plant_months).sum()
june = pd.get_dummies(plant_months).loc[:,6]
june = june.rename("June")
may = pd.get_dummies(plant_months).loc[:,5]
may = may.rename("May")
df = pd.concat([df, may], axis=1)
df = pd.concat([df, june], axis=1)

In [ ]:
print(df.columns)

In [ ]:
pd.get_dummies(df['Season']).sum()

In [ ]:
print(df.columns)
print(df.isnull().sum())

In [ ]:
# LATITUDE AND LONGITUDE CLUSTERING INTO FEATURES

from sklearn.cluster import KMeans

latlong = df.loc[:, ['Latitude', 'Longitude']]

kmeans = KMeans(n_clusters=4, random_state=0).fit(latlong)
kmeans.labels_.shape
lat_long_dummies = pd.get_dummies(kmeans.labels_)
lat_long_dummies = lat_long_dummies.rename(index=int, columns={0: "Loc Clust 0",
                                                               1: "Loc Clust 1",
                                                               2: "Loc Clust 2",
                                                               3: "Loc Clust 3"})
df = pd.concat([df, lat_long_dummies], axis = 1)

In [ ]:
np.asarray(df.iloc[:, df.columns.str.match('V\d\d\d\d\d\d')].columns)

In [ ]:
#THIS IS A VISUALIZATION FOR LATITUDE AND LONGITUDE CLUSTERING

cent = kmeans.cluster_centers_
clust_labels = kmeans.labels_
means = pd.DataFrame(clust_labels)

import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111)
scatter = ax.scatter(df['Latitude'],df['Longitude'],
                     c=means[0], s=50)
ax.set_title('K-Means Clustering')
ax.set_xlabel('Latitude')
ax.set_ylabel('Longitude')
plt.colorbar(scatter)

In [ ]:
#REMOVE ANY NAN VALUES

print(df.columns)
df = df[~df.Silt.isnull()]
df = df[~df['Loc Clust 1'].isnull()]

In [ ]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
binarized = lb.fit(df.Variety)
print(binarized)
df.Variety = pd.Series(binarized.transform(df.Variety))
print(binarized.transform(df.Variety).reshape(1,-1))

In [ ]:
for col in df.columns:
    print(col, type(df[col][0]))

In [7]:
# DROP ALL THE CELLS THAT ARE NOT USABLE SUCH AS THE ONES THAT ARE STRINGS OR DATES

# set if want to drop some columns specifically
should_drop = 1
columns_to_drop = ['Experiment', 'Location',
                   'Check Yield', 'Yield difference', 'Latitude',
                   'Longitude', 'Variety', 'PI', 'Planting date']

# set if want to keep some columns specifically
should_keep = 0
# columns_to_keep = ['Loc Clust 0', 'Loc Clust 1', 'Loc Clust 2', 'Loc Clust 3']
columns_to_keep_top = ['Silt', 'Precipitation', 'Temperature', 'Solar Radiation', 'Organic matter']
columns_VARIETIES_ONLY = np.asarray(df.iloc[:, df.columns.str.match('V\d\d\d\d\d\d')].columns)

#set the below variable to whatever columns you want to keep
columns_to_keep = columns_to_keep_top

MUST_HAVE_COLUMNS = ['Yield']
# print(columns_to_keep)

df = df.drop(columns_to_drop, axis=1) if should_drop else df
df = df.loc[:, np.concatenate((columns_to_keep, MUST_HAVE_COLUMNS))] if should_keep else df
df['YieldBucket'] = pd.Series(pd.qcut(df.Yield, q=3, labels=["high", "medium", "low"]))
print("The final dataframe has columns: ", df.columns)

The final dataframe has columns:  Index(['Yield', 'Year', 'Temperature', 'Precipitation', 'Solar Radiation',
       'Soil class', 'CEC', 'Organic matter', 'pH', 'Clay',
       ...
       'V156574', 'V156642', 'V156763', 'V156774', 'V156783', 'V156786',
       'V156797', 'V156806', 'V156807', 'YieldBucket'],
      dtype='object', length=188)


In [ ]:
print(pd.get_dummies(df.YieldBucket).sum())

In [ ]:
# LET US ALSO MAKE SURE THERE ARE NO NAN IN THE DATA
print("We expect to be %s nan values and there actually are %s nan values\n" % (0, np.sum(df.isnull().sum())))
print(df.isnull().sum())
# AFTER COLUMNS, MAKE SURE NO SKETCHY ONES
for col in df.columns:
    print(col, type(df[col][0]))    

In [8]:
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT

X = df.drop(['Yield', 'YieldBucket'], axis=1)

print(X.columns)

y = df.Yield

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, train_size = 0.1, random_state = 42)

INPUT_COLS = X_train.columns
# TEST_COLS = y_train.columns

Index(['Year', 'Temperature', 'Precipitation', 'Solar Radiation', 'Soil class',
       'CEC', 'Organic matter', 'pH', 'Clay', 'Silt',
       ...
       'V156565', 'V156574', 'V156642', 'V156763', 'V156774', 'V156783',
       'V156786', 'V156797', 'V156806', 'V156807'],
      dtype='object', length=186)


In [5]:
print("X_train shape:", X_train.shape, "\ny_train shape:", y_train.shape)

X_train shape: (8203, 186) 
y_train shape: (8203,)


In [104]:
# This function will evaluate the errors based on RMSE (from the challenge spec)
# also will evaluate based on average error

from sklearn.metrics import mean_squared_error
def evaluate_errors(prediction, actual):
    print("RMSE Error: ", np.sqrt(mean_squared_error(prediction, actual)))
    avg_error_vector = np.absolute(((preds - y_test) / y_test) * 100)
#     print("Average Error: ", np.mean(avg_error_vector))
    print("Average Error details:\n", avg_error_vector.describe())
    return avg_error_vector

In [105]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_estimators=20, max_depth=13, random_state=0, verbose=1)
regr.fit(X_train, y_train)
preds = regr.predict(X_test)

evaluate_errors(preds, y_test)


RMSE Error:  7.222372654111693
Average Error details:
 count    4102.000000
mean       10.064586
std        10.488447
min         0.000290
25%         3.567128
50%         7.447855
75%        13.134655
max       166.409689
Name: Yield, dtype: float64


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished


76141    11.269207
11321    32.332536
68096     1.308601
10813     5.441149
30586    11.259631
62263     1.748256
58403    10.817987
45601    23.555064
69170    13.344366
20865     0.352562
14592    15.906424
79035     0.079519
48667    11.429723
31714     7.077961
48739     0.345438
73832     3.669843
38065    14.414044
17050    15.660528
61447     6.669599
9293      0.205083
64269     3.729395
50211     4.265225
15219     5.392042
43234    10.003270
60842     9.700697
33703     0.076046
40339     6.406243
78791    16.681696
54940    12.780832
32595     3.281516
           ...    
47740     0.900416
35395     4.621874
54690    20.425645
58371     4.372042
41131     8.123907
48939     5.660572
71954     0.298012
80128     0.124216
23210    19.457195
59380     9.762432
49343     3.909380
75084     3.909873
3821      4.506140
27294    14.271453
56009    11.919151
8107      0.461441
4086      0.242372
2522     14.913449
81788     2.452257
3024      5.001782
71351    12.886050
16141    23.

In [ ]:
pd.get_dummies(df['Plant date']).sum()

In [ ]:
# GET OUTPUT OF FEATURE IMPORTANCE
def get_feature_importances(regr):
    feature_importances = regr.feature_importances_
    feature_importances = pd.Series(feature_importances)
    feature_importance_df = pd.DataFrame({'feature': X_train.columns,'feature_importance': feature_importances})
    feature_importance_df = feature_importance_df.sort_values(by=['feature_importance'])
    for index, row in feature_importance_df.iterrows():
        print(row['feature'], 'has importance: ', row['feature_importance'])
get_feature_importances(regr)

In [ ]:
# THIS WILL ONLY WORK WITH THE BUCKET METHOD

from sklearn.ensemble import RandomForestClassifier
regr = RandomForestClassifier(n_estimators=10, max_depth=20, random_state=0, verbose=1)
regr.fit(X_train, y_train)
preds = regr.predict(X_test)

from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, preds))

In [ ]:
import numpy as np
from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPRegressor

from sklearn.feature_selection import RFECV

classifiers = [
    svm.SVR(),
    MLPRegressor(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(5, 2), random_state=1),
    linear_model.SGDRegressor(),
    linear_model.BayesianRidge(),
    linear_model.LassoLars(),
#     linear_model.ARDRegression(),
#     linear_model.ARDRegression(),
    linear_model.PassiveAggressiveRegressor(),
    linear_model.TheilSenRegressor(),
    linear_model.LinearRegression()]




# estimator = svm.SVR(kernel="linear")

# selector = RFECV(estimator, step=1, cv=5, verbose=1)
# selector = selector.fit(X_train, y_train)
# selector.support_ 
# # array([ True,  True,  True,  True,  True,
# #         False, False, False, False, False], dtype=bool)
# selector.ranking_
# # array([1, 1, 1, 1, 1, 6, 4, 3, 2, 5])


#     print(np.sum(preds - y_test))
#     print(clf.predict(X_test),'\n')
#     print(y_test)
#     print('accuracy score:', accuracy_score(y_test, clf.predict(X_test)), '\n')


In [ ]:
for item in classifiers:
    print(item)
    clf = item
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    errors = evaluate_errors(preds, y_test)
    try:
        get_feature_importances(clf)
    except:
        print("NO FEATURE IMPORTANCE METRIC")
#     errors = np.absolute(((preds - y_test) / y_test) * 100)
#     print(errors)
#     print(errors)
#     print(errors.describe())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifiers = [
#     KNeighborsClassifier(3),
#     SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
#     GaussianProcessClassifier(1.0 * RBF(1.0)),
#     DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]
from sklearn.metrics import accuracy_score
for item in classifiers:
    print(item)
    clf = item
    clf.fit(scale(X_train), y_train)
    preds = clf.predict(scale(X_test))
    print(accuracy_score(y_test, preds))
#     errors = np.absolute(((preds - y_test) / y_test) * 100)
#     print(errors)
#     print(np.mean(errors))

In [202]:
import random

NUM_VARIETIES = 174

def best_yield_variety(regr, test_set, random_sel = True, n_samples = 174, print_variety_preds = True):
    
    #create empty df
    dup_df = pd.DataFrame()
    
    #choose a rand sample of input test_set (for dev purposes, wouldn't be used in app)
    test_set_sample = test_set.sample(n= n_samples) if random_sel else test_set
    
    #progress of intensive, long for loop upcoming
    counter = 0
    
    #for loop will, for each row in test_set_sample, duplicate that row by NUM_VARIETIES
    for index, row in test_set_sample.iterrows():
        counter+=1
        print(counter)
        dup_df = dup_df.append([row] * NUM_VARIETIES, ignore_index = True)
        
    #extract the varieties columns
    duplicated_df_varieties = dup_df.loc[:, dup_df.columns.str.match('V\d\d\d\d\d\d')]
    #extract the names of the varieties
    varieties_array = duplicated_df_varieties.columns
    num_expanded_data_pts = duplicated_df_varieties.shape[0]
    #we must have a zeroed matrix of the same shape as the duplicated_df_varieties
    #so that we can input a 1 just once for each variety per 174 rows
    d = np.zeros((duplicated_df_varieties.shape[0], NUM_VARIETIES))
    #make d our dataframe, with columns equal to the varieties
    duplicated_df_varieties = pd.DataFrame(d, columns=varieties_array)
    #for loop will place a 1 just once for each variety per 174 rows (one hot rep)
    for i in range(duplicated_df_varieties.shape[0]):
        var_index = i % 174
        duplicated_df_varieties.loc[i, varieties_array[var_index]] = 1
    #remove the varieties (will be added back with the new values)
    dup_df = dup_df.drop(varieties_array, axis = 1)
    #add the new values (one hot representations) from above for loop
    dup_df = pd.concat([dup_df, duplicated_df_varieties], axis=1)
    #do prediction on the entire dataframe
    preds_per_variety = regr.predict(dup_df)
    #*******make it into a dataframe where each row will give the performance of each variety
    #with the same environmental conditions*******
    preds_df = pd.DataFrame(preds_per_variety.reshape((int(num_expanded_data_pts/NUM_VARIETIES), NUM_VARIETIES)),
                            columns=varieties_array)
    
    #environmental conditions (everything except the variety data)
    envcond = test_set_sample.drop(varieties_array, axis=1)
    
    #a simple print out for best variety given the environmental conditions
    hr_preds = []
    if print_variety_preds:
        envcond_cols = envcond.columns
        counter = -1
        for idx, row in envcond.iterrows():
            counter+=1
            out = "For environmental conditions:\n%s\nthe best variety is:%s" % (row, preds_df.idxmax(axis=1)[counter])
            hr_preds.append(out)
            print(out)
    
    return preds_df, envcond, hr_preds
    
        
preds_df, envcond = best_yield_variety(regr, X_test, n_samples = 174)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished


For environmental conditions:
Year               2.015000e+03
Temperature        3.228800e+03
Precipitation      7.621000e+02
Solar Radiation    1.017930e+06
Soil class         2.120000e+02
CEC                2.491406e+01
Organic matter     5.919804e+00
pH                 6.078154e+00
Clay               2.242506e+01
Silt               4.275890e+01
Sand               3.480946e+01
Area               2.720000e-01
Name: 75835, dtype: float64
the best variety is:V150844
For environmental conditions:
Year               2.010000e+03
Temperature        3.368700e+03
Precipitation      8.185000e+02
Solar Radiation    1.060760e+06
Soil class         1.120000e+02
CEC                2.678516e+01
Organic matter     7.022948e+00
pH                 6.205035e+00
Clay               2.202199e+01
Silt               4.259346e+01
Sand               3.540733e+01
Area               2.283000e-01
Name: 6334, dtype: float64
the best variety is:V103163
For environmental conditions:
Year               2.013000e+03

For environmental conditions:
Year               2.011000e+03
Temperature        3.251100e+03
Precipitation      6.333000e+02
Solar Radiation    1.047271e+06
Soil class         2.120000e+02
CEC                2.491406e+01
Organic matter     5.919804e+00
pH                 6.078154e+00
Clay               2.242506e+01
Silt               4.275890e+01
Sand               3.480946e+01
Area               2.720000e-01
Name: 15482, dtype: float64
the best variety is:V155843
For environmental conditions:
Year               2.009000e+03
Temperature        3.623900e+03
Precipitation      6.165000e+02
Solar Radiation    1.114843e+06
Soil class         2.120000e+02
CEC                2.585453e+01
Organic matter     3.356874e+00
pH                 6.049624e+00
Clay               2.752416e+01
Silt               5.415524e+01
Sand               1.827532e+01
Area               3.271000e-01
Name: 979, dtype: float64
the best variety is:V139548
For environmental conditions:
Year               2.011000e+03


In [212]:
maxes = preds_df.idxmax(axis=1) # get first max variety per env
pd.get_dummies(maxes).sum().sort_values(ascending=False)

V156806    15
V150844    15
V156368    13
V103163    12
V103293    11
V139548    11
V155820     7
V152253     6
V120047     6
V155843     6
V104000     5
V111336     4
V103281     4
V114565     4
V103142     4
V131675     3
V103302     3
V103150     3
V121015     2
V114655     2
V156553     2
V120410     2
V156516     2
V000092     2
V000062     2
V156247     2
V155918     2
V000172     2
V151284     2
V150847     2
V103866     1
V000039     1
V000122     1
V000098     1
V120999     1
V113396     1
V114545     1
V114564     1
V156786     1
V137136     1
V137237     1
V150853     1
V151333     1
V152079     1
V152320     1
V152734     1
V156774     1
V000034     1
dtype: int64

In [198]:
envcond_cols = envcond.columns
hr_preds = []
counter = -1
for idx, row in envcond.iterrows():
    counter+=1
    out = "For environmental conditions:\n%s\nthe best variety is:%s" % (row, preds_df.idxmax(axis=1)[counter])
    hr_preds.append(out)
    print(out)
#     out = "For environmental conditions:\n
#     %s\n, the best variety is: %s" % ([str(envcond_cols[i]) + str(row[i]) for i in range(0, len(row))])
#     print(row[0])

In [201]:
print(hr_preds[0])

For environmental conditions:
Year               2.011000e+03
Temperature        3.626100e+03
Precipitation      6.555000e+02
Solar Radiation    1.057904e+06
Soil class         2.130000e+02
CEC                2.906503e+01
Organic matter     5.236967e+00
pH                 6.404890e+00
Clay               2.774834e+01
Silt               5.493584e+01
Sand               1.735388e+01
Area               3.782000e-01
Name: 18914, dtype: float64
the best variety is:V120047


In [177]:
preds_df.describe().sort_values(by="mean", axis=1)

,V156642,V130308,V155842,V139107,V103198,V000123,V000024,V120246,V152053,V000047,...,V150847,V000062,V150844,V103150,V139548,V111336,V156247,V103293,V156368,V156806
count,870.000000,870.000000,870.000000,870.000000,870.000000,870.000000,870.000000,870.000000,870.000000,870.000000,...,870.000000,870.000000,870.000000,870.000000,870.000000,870.000000,870.000000,870.000000,870.000000,870.000000
mean,58.187644,58.230591,58.302099,58.494142,58.605243,58.616659,58.629732,58.679578,58.742627,58.745234,...,59.421955,59.439755,59.473157,59.474146,59.569037,59.569280,59.652401,59.810733,59.985121,60.213240
std,8.509407,9.517469,8.386367,8.441235,9.090426,9.361030,8.627873,8.714769,8.136712,8.618932,...,8.277774,8.125106,7.944667,8.327004,8.209543,8.549799,8.240195,8.723531,7.953090,8.471066
min,33.726878,33.726878,33.726878,33.726878,33.726878,14.751404,33.726878,33.726878,33.726878,33.726878,...,33.726878,33.726878,33.726878,33.726878,36.873257,33.726878,33.726878,33.726878,33.726878,33.726878
25%,52.046471,52.277383,52.346966,52.277383,52.864703,52.797721,52.823103,53.033071,52.823103,52.277383,...,52.823103,53.553578,53.853685,53.043123,53.033071,53.502006,53.612357,53.553578,54.369756,53.898564
50%,58.436219,59.288090,59.194860,58.226302,59.250880,59.194860,58.827094,59.250880,59.194860,59.255896,...,60.059664,59.320284,59.359836,59.253489,60.431843,59.255896,60.292099,59.564227,59.808186,60.092972
75%,64.539777,65.316277,64.695166,65.308259,65.316277,65.316277,65.308259,65.314272,64.565552,65.316277,...,65.411011,65.411011,65.316277,65.681424,66.241739,66.332446,65.411011,66.751553,66.247793,66.245984
max,93.153255,93.153255,86.969411,93.153255,93.153255,93.153255,93.153255,93.153255,93.153255,93.153255,...,93.153255,93.153255,93.153255,101.860192,93.153255,106.341387,93.153255,93.153255,98.975575,93.153255


In [42]:
import random
duplicated_df = pd.DataFrame()
counter = 0
X_test_random_rows = X_test.sample(n=174)
# print(X_test_random_rows)
for index, row in X_test_random_rows.iterrows():
    counter+=1
    print(counter)
    duplicated_df = duplicated_df.append([row]*174, ignore_index=True)
    
#     print([row]*2)
#     break
#     print(X_test[row.name])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174


In [43]:
duplicated_df_varieties = duplicated_df.loc[:, duplicated_df.columns.str.match('V\d\d\d\d\d\d'))]

In [44]:
duplicated_df_varieties.shape

(30276, 174)

In [62]:
ddv = duplicated_df_varieties

In [70]:
d = np.zeros((30276, 174))

In [71]:

duplicated_df_varieties = pd.DataFrame(d, columns = duplicated_df_varieties.columns)

In [73]:
varieties_array = duplicated_df_varieties.columns

In [74]:
print(varieties_array.shape)

(174,)


In [76]:
ddv = duplicated_df_varieties

In [86]:
ddv.loc[0, varieties_array[0]] = 0
# varieties_array[3]
ddv.sum()

V000016    0.0
V000017    0.0
V000018    0.0
V000023    0.0
V000024    0.0
V000025    0.0
V000030    0.0
V000032    0.0
V000034    0.0
V000036    0.0
V000039    0.0
V000047    0.0
V000050    0.0
V000051    0.0
V000058    0.0
V000060    0.0
V000062    0.0
V000067    0.0
V000070    0.0
V000071    0.0
V000075    0.0
V000078    0.0
V000079    0.0
V000080    0.0
V000081    0.0
V000082    0.0
V000092    0.0
V000096    0.0
V000098    0.0
V000110    0.0
          ... 
V152253    0.0
V152300    0.0
V152312    0.0
V152320    0.0
V152322    0.0
V152440    0.0
V152734    0.0
V152779    0.0
V155180    0.0
V155820    0.0
V155842    0.0
V155843    0.0
V155918    0.0
V156247    0.0
V156305    0.0
V156314    0.0
V156367    0.0
V156368    0.0
V156516    0.0
V156553    0.0
V156565    0.0
V156574    0.0
V156642    0.0
V156763    0.0
V156774    0.0
V156783    0.0
V156786    0.0
V156797    0.0
V156806    0.0
V156807    0.0
Length: 174, dtype: float64

In [87]:
duplicated_df_varieties.sum()

for i in range(duplicated_df_varieties.shape[0]):
    var_index = i % 174
    duplicated_df_varieties.loc[i, varieties_array[var_index]] = 1

In [97]:
ddv

,V000016,V000017,V000018,V000023,V000024,V000025,V000030,V000032,V000034,V000036,...,V156565,V156574,V156642,V156763,V156774,V156783,V156786,V156797,V156806,V156807
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [94]:
duplicated_df = duplicated_df.drop(varieties_array, axis = 1)

In [98]:
duplicated_df = pd.concat([duplicated_df, ddv], axis=1)

In [103]:
print(duplicated_df.loc[0])

Year               2.009000e+03
Temperature        2.913400e+03
Precipitation      5.704000e+02
Solar Radiation    1.034483e+06
Soil class         1.110000e+02
CEC                2.670233e+01
Organic matter     1.041580e+01
pH                 6.707646e+00
Clay               2.279022e+01
Silt               4.360800e+01
Sand               3.366537e+01
Area               2.030000e-02
V000016            1.000000e+00
V000017            0.000000e+00
V000018            0.000000e+00
V000023            0.000000e+00
V000024            0.000000e+00
V000025            0.000000e+00
V000030            0.000000e+00
V000032            0.000000e+00
V000034            0.000000e+00
V000036            0.000000e+00
V000039            0.000000e+00
V000047            0.000000e+00
V000050            0.000000e+00
V000051            0.000000e+00
V000058            0.000000e+00
V000060            0.000000e+00
V000062            0.000000e+00
V000067            0.000000e+00
                       ...     
V152253 

In [ ]:
def yield_per_variety(clf, env_conditions):
    

In [110]:
ppv = regr.predict(duplicated_df)

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished


In [115]:
pd.Series(ppv[1:174]).describe()

count    173.000000
mean      52.976309
std        0.765295
min       46.502349
25%       53.069376
50%       53.069376
75%       53.069376
max       55.164769
dtype: float64

In [118]:
len(np.unique(ppv[0:174])) #29 unique predictions...

29

In [120]:
preds_df = pd.DataFrame(columns=varieties_array)

TypeError: cannot concatenate object of type "<class 'numpy.ndarray'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid

In [129]:
ppvv = ppv.reshape(174,174)

In [130]:
preds_df = pd.DataFrame(ppvv, columns=varieties_array)

In [147]:
preds_df.sum().sort_values()

V130308    10038.657542
V156642    10040.539596
V103198    10070.863269
V155842    10096.764758
V152053    10127.423327
V120246    10136.043622
V000024    10137.194764
V139107    10145.100898
V151399    10145.793446
V103163    10146.955972
V000123    10149.196568
V155843    10151.285259
V000025    10163.836784
V150834    10179.386901
V103132    10183.350854
V000047    10184.324654
V114564    10186.589756
V136868    10186.996247
V000060    10187.598789
V156305    10188.192017
V000039    10188.260916
V151036    10189.450078
V103620    10194.724018
V000018    10196.706346
V120810    10196.716245
V103156    10197.637201
V150853    10198.880134
V103425    10201.783624
V137147    10202.707081
V120038    10204.818743
               ...     
V156314    10239.413105
V000092    10239.912155
V121015    10240.203521
V151333    10242.887665
V103281    10244.387201
V137136    10244.443349
V156797    10244.813543
V113396    10246.800860
V000172    10247.434413
V152734    10253.601245
V156565    10254